<a href="https://www.kaggle.com/code/shahjaysuhasbhai/text-summarizatio?scriptVersionId=273131298" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 15.1 MB/s eta 0:00:0000:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

1. AutoTokenizer: class from the Hugging Face Transformers library
2. from_pretrained(model_name): downloads particular tokenizer identified by model_name
3. AutoModelForSeq2SeqLM: class from Hugging Face’s transformers library designed for sequence-to-sequence tasks, such as translation, text summarization, etc
4. from_pretrained(model_name): downloads the model identified by model_name with **weights and architecture** of it.

In [4]:
model_name = "facebook/bart-large-cnn"  # or "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

2025-11-03 12:19:52.815813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762172393.096959      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762172393.193926      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

     inputs = tokenizer(text, return_tensors="pt", max_length=max_input_length, truncation=True)

1. tokenizer: converts raw text into token representations
2. return_tensors="pt": output will be in pytorch tensors as model should require tensors, not tokens-strings
3. max_length=max_input_length, Here, 1024 tokens
4. truncation = True means if text has more than 1024 tokens then it truncates them

In [13]:
# takes input text, encodes and summarizes it.
def summarize(text, model, tokenizer, max_input_length=1024, max_output_length=100):
    #inputs contains text with it's corresponding vectored representation for each tokens
    inputs = tokenizer(text, return_tensors="pt", max_length=max_input_length, truncation=True)
    summary_ids = model.generate(
        inputs["input_ids"], # input_ids having vectored/tensor representation
        num_beams=4, 
        max_length=max_output_length, 
        early_stopping=True #  generation process will stop once the beam search finds a complete hypothesis for all beams, not continuing generation unnecessarily 
    )
    # summary_ids[0] contains token_ids generated by model, we have to decode it
    # skip_special_tokens=True will skip special tokens like <b> <s> > ? / etc
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True) 
    return summary


**num_beams=4** : Beam search is a text generation algorithm that tries to find the most likely output sequence by exploring multiple candidate sequences simultaneously.
1. with num_beams=4, at each generation step the algorithm keeps the 4 most promising candidate sequences.
2. When num_beams=1, beam search reduces to greedy search, only considering the single best next token at every step.

In [14]:
article = """
A hungry crow found a piece of bread and flew away with it. As the crow settled on a tree branch, 
a clever fox saw the bread and desired it. The fox praised the crow for its beautiful wings and sweet voice, 
flattering it to make the crow sing. Wanting to impress the fox, the crow opened its beak to caw, dropping the bread.
The fox quickly grabbed the bread and disappeared into the woods, leaving the crow to reflect on its mistake.
"""
summary_result = summarize(article, model, tokenizer)
print(summary_result)


A fox praised the crow for its beautiful wings and sweet voice. Wanting to impress the fox, the crow opened its beak to caw, dropping the bread. The fox quickly grabbed the bread and disappeared into the woods, leaving the crow to reflect on its mistake.
